In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
train

In [ ]:
sns.catplot(x="Parch", hue="Survived", col = 'Sex', data=train, kind = 'count')

In [ ]:
#TRAIN
train_new = train
#Rellenar Age con la media
Age = train['Age']
Age_new = Age.fillna(Age.mean())
train['Age'] = Age_new
#Separamos Embarked en columnas de 0 o 1
Embarked_new = train_new['Embarked']
Embarked_new.fillna('S')
Embarked_new = pd.get_dummies(Embarked_new)
train_new = pd.concat([train_new, Embarked_new], axis=1)
train_new.drop(columns = 'Embarked', inplace = True)
#Separamos Sex en 0 o 1
Sex_new = train_new['Sex']
Sex_new = pd.get_dummies(Sex_new)
Sex_new.drop(labels = ['male'],axis = 1, inplace = True)
train_new['Sex'] = Sex_new

X = np.array(train_new.drop(['Survived','Name','Ticket','Cabin'],1))
Y = np.array(train_new['Survived'])

#TEST
#Age y Fare con la media, y lo mismo con Embarked
Age = test['Age']
Age_new = Age.fillna(Age.mean())
test['Age'] = Age_new
Fare = test['Fare']
Fare_new = Fare.fillna(Fare.mean())
test['Fare'] = Fare_new
test_new = test
Embarked_new = test_new['Embarked']
Embarked_new.fillna('S')
Embarked_new = pd.get_dummies(Embarked_new)
test_new = pd.concat([test_new, Embarked_new], axis=1)
test_new.drop(columns = 'Embarked', inplace = True)
Sex_new = test_new['Sex']
Sex_new = pd.get_dummies(Sex_new)
Sex_new.drop(labels = ['male'],axis = 1, inplace = True)
test_new['Sex'] = Sex_new

X_test = np.array(test_new.drop(['Name','Ticket','Cabin'],1))

In [ ]:
#https://towardsdatascience.com/beginners-guide-to-xgboost-for-classification-problems-50f75aac5390
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [5, 7, 10],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

xgb_cl = xgb.XGBClassifier()
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")
_ = grid_cv.fit(X, Y)

In [ ]:
xgb_model = xgb.XGBClassifier(grid_cv.best_params_,colsample_bytree=0.5,
    subsample=0.8)

xgb_model.fit(X, Y)

In [ ]:
X_test = np.array(test_new.drop(['Name','Ticket','Cabin'],1))

In [ ]:
Y_pred_test = xgb_model.predict(X_test)
test_new['Survived'] = Y_pred_test
final_predict = test_new[['PassengerId','Survived']]
final_predict.to_csv(path_or_buf = r'./Titanic_XGB_predict.csv', index = False)